# 进程和线程

### 概念
#### 进程
指OS(操作系统)中执行的一个程序的实例,OS以进程为资源分配和调度的基本单位,程序只是指令,数据及其组织形式的描述,而进程是程序的实体.
1. 每个进程有自己**独立**的`地址空间`和`数据栈`.
2. 进程可以通过`fork`或者`spawn`的方式创建新的进程.
3. 进程间通过`IPC`(inter-process communication)实现数据共享,具体包括管道,信号,套接字,共享内存区等方法.
#### 线程
一个进程还可以拥有多个并发的执行线索,(???什么鬼),即拥有多个可以获得CPU调度的执行单元,这就是**线程**.
每个进程可以有多个线程,由于多个线程同属于一个进程,所以他们的**资源共享,通信共享**.